# Computing and parsing multiple-sequence alignments
(Víctor Sojo | vsojo@amnh.org)

In the previous lesson we created a FASTA file with four sequences for the gene encoding the red panda's 12S ribosomal RNA.

In this notebook:
1. We will initially move beyond **BioPython** to install alignment software (MAFFT and Clustal Omega) with `conda`.
1. We will then use this software to align the red panda sequences through the terminal – all of which we can do without leaving Jupyter.
1. We will then read the alignments using BioPython, and explore some (but certainly not all) of the properties of BioPython alignment module, `AlignIO`, which includes tools for reading, writing, slicing, joining, and counting substitutions in an alignment.
1. The red panda sequences are too similar to be of any interest, so we will download and align a richer set of 12S rRNA sequences, so that we can use them to build a phylogenetic tree in the following lesson.

**References and suggested reading:**
+ The [_BioPython tutorial_](http://biopython.org/DIST/docs/tutorial/Tutorial.html) could use some updating, but it's very useful if you have a decent idea of what you're looking for.

## Contents
&emsp;[Importing required BioPython modules](#Importing-required-BioPython-modules)<br/>
&emsp;[Inspecting the multiple-sequence FASTA file](#Inspecting-the-multiple-sequence-FASTA-file)<br/>
&emsp;[Installing new software via conda](#Installing-new-software-via-conda)<br/>
&emsp;[Aligning sequences](#Aligning-sequences)<br/>
&emsp;[Aligning more interesting sequences](#Aligning-more-interesting-sequences)<br/>
&emsp;[Loading and processing alignments with BioPython's AlignIO module](#Loading-and-processing-alignments-with-BioPython's-AlignIO-module)<br/>
&emsp;[Some useful AlignIO methods](#Some-useful-AlignIO-methods)<br/>
&emsp;&emsp;[Use \[:\] notation to get only some sequences of the alignment](#Use-\[:\]-notation-to-get-only-some-sequences-of-the-alignment)<br/>
&emsp;&emsp;[Use double \[ : , : \] notation to get \"slices\" of an alignment](#Use-double-\[-:-,-:-\]-notation-to-get-\"slices\"-of-an-alignment)<br/>
&emsp;&emsp;[Use + to join desired slices into a Frankensteined alignment](#Use-+-to-join-desired-slices-into-a-Frankensteined-alignment)<br/>
&emsp;[Use AlignIO.write\(\) to write an alignment to a file](#Use-AlignIO.write\(\)-to-write-an-alignment-to-a-file)<br/>
&emsp;[Use AlignIO.write\(\) to convert between formats](#Use-AlignIO.write\(\)-to-convert-between-formats)<br/>
&emsp;&emsp;[Explore changes between sequences with the .substitutions attribute](#Explore-changes-between-sequences-with-the-.substitutions-attribute)<br/>

Once again, let's make sure that we're using the `bioinfo` environment that we created in the `Py201_` notebook:

(If you're on Windows, remember that every line with `! some code` should be changed to `!wsl some code` and you should have an active [WSL installation](https://docs.microsoft.com/en-us/windows/wsl/install-win10))

## Importing required BioPython modules
Here we will need:

Module        | Use
:-------------|:-----------------------------------------
**SeqIO**     | To handle parsing, reading and writing sequences
**AlignIO**   | Similar to SeqIO, but for alignments
**SeqRecord.SeqRecord** | To create new sequence records

## Inspecting the multiple-sequence FASTA file
In the previous lesson, we created a common fasta file that contains the sequences of all 4 12S rRNA gene entries that we found for red pandas on NCBI, using our simple literal search (we could have performed a BLAST search instead). Let's take a quick look at the FASTA identifier line for each of the four sequences, as well as the following line (that's what `-A1` is for), which contains the first line of the DNA sequence for each gene:

Now we can use `grep -A1` to check out the sequence identifier and first line of sequence:

---
That looks good. What we want to do here is align this file so that we can compare the four sequences to check if they are the same or not.

## Installing new software via `conda`
To align the sequences, we need a sequence aligner. There are multiple options here, but two common ones are Clustal Omega (better know to its friends as `clustalo`) and MAFFT (typically known as `mafft` but sometimes `mafft-linsi` or other names).

There are multiple ways to get these two packages. Most intuitively, you could just go to their websites, download from there and follow their installation instructions. That works fine, and it's what many bioinformaticians do. However, `conda` is great in that it not only manages Python packages, it also manages regular software such as `blast+`, `clustalo`, `mafft`, `iqtree`, `fasttree`, and so many others. So, you can just install each of these as a `conda` package and let `conda` deal with any dependencies and future updates.

It's beautiful, and you can even do it from within Jupyter:

---
⚠️ **This will not work on Windows!** ⚠️<br/>
Unfortunately, conda can't yet communicate with the WSL as properly as it perhaps should; so, at least until my latest attempts, it was not possible to install linux-shell software such as `mafft` or `clustalo`. For this reason, `conda` on Windows will tell you that the packages above do not exist. They do, but they can't be installed under Windows through `conda`. There are several options for you. For example, you could try to follow the installation instructions for MAFFT in Ubuntu under Windows [here](https://mafft.cbrc.jp/alignment/software/ubuntu_on_windows.html) and see if that works.

But if that fails, I have included links below to download the files instead of computing them (sorry!).

---
Anyhow, moving on...

I already have both packages installed, so not much happens in my case; you may have to wait for a short while `conda` checks for updates or downloads and installs.

Let's take a look at which versions we got:

From skimming the official [Clustal](http://www.clustal.org/omega/) and [MAFFT](https://mafft.cbrc.jp/alignment/software/) websites, those are the latest versions at the time of writing; great!

Even better, if they update, we won't need to bother checking on their websites; we can just periodically (every couple of weeks or so) open a new terminal window and run:
```bash
conda update --all -y
conda update --all -y -n bioinfo
```
And all of conda and our `bioinfo` environment will update, including `clustalo`, `mafft`, `blast+`, `jupyter`, `conda` itself, and whatever else we have installed. It's wonderful.

More and more software developers are noticing how powerful this is and releasing their packages to conda, either directly or via additional channels (such as `conda-forge` and `bioconda`, which we added in the `Py201` notebook).

So, **before you install new software the traditional way (i.e. by downloading it from their website), first check if it is available through `conda`**
One way to do that is through the [search function on the Anaconda website](https://anaconda.org/).

---

⚠️ **Windows warning** ⚠️ As mentioned above, many packages available through `conda` run not through Python, but through the shell, so they won't work on Windows (yet), even if you have WSL.

---

Let's print the help for `mafft`:

---
It seems the simplest way to run `mafft` is to issue the following in a terminal:
```python
mafft unaligned_seqs.fasta > aligned_seqs.fasta
```
But if you want higher accuracy and have fewer than ~200 sequences with fewer than 2,000 monomers each, you can use other options. We will stick to the simpler fast version for now. Read the manual for higher accuracy.

## Aligning sequences
There are multiple ways to access the terminal in Python, and if you were writing a normal Python script, you would want to look into [the **subprocess** module](https://docs.python.org/3/library/subprocess.html). However, as we have seen before, mighty Jupyter can access the terminal directly by simply specifying a `!` (or `!wsl` in Windows), so we won't even bother trying to run software through Python here when we can just use Jupyter.

We want to align our unaligned fasta file. Let's first take a quick look at its actual name again, which we have stored in a variable:

Let's use that name to create a new identical name, but with `'unaligned'` changed to `'mafft'`, so that we later know that we aligned it using `mafft`:

Great. Now we can just use these two variable names to run `mafft`:<br/>
(⚠️ Windows users: skip the following, you should have the aligned file pre-computed.)

I don't see any complaints, so this is probably all good.

Let's see if it worked by printing the fasta sequence identifier line (starts with `>`) and the next line after that (using `-A1`) for each of the 4 sequences:

Looking good! The dashes `-` are the places in which `mafft` inserted either gaps or padding at the ends to make the sequences match.

One nice software to visualise alignments is [SeaView](http://doua.prabi.fr/software/seaview), which unfortunately does not seem to have a `conda` package (yet).

I have loaded the aligned file into my local SeaView, taken a screenshot, and pasted it below – which incidentally lets me show you how to add images to a markdown cell in a Jupyter Notebook:
![Screenshot of a multiple-sequence alignment in SeaView](https://github.com/vsojo/Python_Workshop/raw/DataAnalysis/RedPanda_12SrRNA_SeaView_capture.png "MAFFT alignment of four 12S rRNA genes from the red panda, in SeaView.")
(Note that here I put the image on my GitHub so that it's accessible online through a URL. You could just use local images instead if you want, but then you'd have to send them with your Jupyter Notebook when sharing your code with colleagues, otherwise the image links will break).

Here I'm only showing a fragment of the multiple-sequence alignment, specifically between positions 430 and 600. As you can see, the sequences are essentially the same, except that the second one is missing quite a chunk. We won't go into the biology or molecular genetics of that. Suffice to say that this is not a particularly interesting alignment (other than to visually inspect that they are all indeed the same sequences). We will therefore leave it at this and won't pursue this alignment further.

We will instead download and align a richer set of 12S rRNA sequences from several mammals.

## Aligning more interesting sequences
Let's use 12S rRNA sequences from several different mammals – instead of all from the red panda – so that we can align them and later build a phylogenetic tree with them to see which species are closer to which (at least using this one tiny gene as reference).

I will not go here into how you should acquire orthologues. For now, I have a made file with multiple FASTA sequences for 12S rRNA from several mammals that we will download from my GitHub:

This is a `fasta` file. Let's take a look at all the sequence identifiers (which start with `>`. Open the file in a text editor if you're not familiar with FASTA files):

There are `23` sequences in the file, and you'll have to trust me that they are all acceptable DNA sequencees encoding the 12S rRNA for each of those species.

Let's align the sequences using MAFFT:

I have again loaded this file into SeaView, taken a screen capture, and put that on my GitHub, so that we can load it here:

![Screenshot of a multiple-sequence alignment in SeaView](https://raw.githubusercontent.com/vsojo/Python_Workshop/DataAnalysis/Mammals_12SrRNA_SeaView_capture.png "MAFFT alignment of DNA sequences encoding 12S rRNA from 17 mammals.")

That looks like a much more interesting alignment. Let's load it in using BioPython's AlignIO module.

## Loading and processing alignments with BioPython's `AlignIO` module
BioPython has a specific tool for dealing with alignments. It's called `AlignIO` and we imported it at the beginning of this Notebook (I typically like to import all pakcages at the beginning of my code, but some programmers prefer to import packages only right before they need to use them. You'll see both styles frequently).

Let's use `AlignIO` to read in our freshly created alignment, whose name is still stored in the `aligned_file` variable:

Just a simple `print` is already very helpful. It seems to print only selected sequences (seemingly the first 18, plus the last one), but it does give a very good general picture.

`AlignIO` can do a lot more. Let's explore a bit.

## Some useful `AlignIO` methods

### Use `[:]` notation to get only some sequences of the alignment
Using the traditional `[:]` notation from Python will choose sequences in the order in which they are entered into the alignment. For example, above we have `Human` first, `GiantPanda` second... `Koala` fourth. Remembering that Python starts counting at zero and subsets until one-before-last:

Positions `1` to `3` (remember that `4` is left out) are the Giant Panda, Red Panda, and Koala. This is useful if we want to look only at some sequences but not others.

### Use _double_ `[ : , : ]` notation to get "slices" of an alignment
We can get fractions of an alignment, sliced vertically, by using a double `[:,:]` notation. The first `:` works as before, choosing sequences, whereas the second one chooses vertically from which to which position in the alignment you want to see. More specifically, it would be `[start_seq:end_seq , start_pos:end_pos]`.

Do remember that if you leave a Python subset empty (`[:]`) without specifying any limits, that just means "everything", i.e. from the beginning to the end.

For example, let's get a slice for all sequences, between the 5th and 25th position of the alignment:

We can also get just one position. For example, position `31` (fourth one from the end in the last printout) has quite a few variations between the different sequences. We can take a look at it on its own by specifying a single index to the slicer (i.e. the second position in the `[,]` block):

Check against the full alignment and you'll see those are the nucleotides present at position `31` for each of our `23` species.

⚠️ Note that `AlignIO` positions are for the alignment, not for the sequences themselves.

### Use `+` to join desired slices into a Frankensteined alignment
If you have any reason to remove fragments from your alignment, you can easily do so by adding together the parts that you do want to keep, using `+`. For example, let's say we want the first `5` and last `7` columns of the alignment, but nothing else:

This of course has no biological meaning whatsoever, but you may encounter cases in which you need to do such manipulations, for example, if you want to keep all the water-exposed portions of a protein, or all the (non)repetitive regions of a gene. You will need some biologically sound way of deciding what you want to keep and why (which can be far from trivial), but it's good to know that BioPython makes it so easy to manipulate the sequences.

## Use `AlignIO.write()` to write an alignment to a file
Let's say for some reason we want to keep that weird fragment with 12 Frankensteined columns. We can easily do that with `AlignIO.write()`.

Check that it worked:

## Use `AlignIO.write()` to convert between formats
Because BioPython makes it so easy to both read and write alignment files, we could trivially combine these two strengths to convert between file formats.

We can use this approach to convert our `fasta` alignments to `phylip` format:

Let's take a look at the `phylip` file:

That was easy!

The same trick applies to `SeqIO`, which you can use to convert between file formats of non-aligned files.

We don't need either of those frankensteined files, though, so let's delete them before we mistakenly use them for something else:

### Explore changes between sequences with the `.substitutions` attribute

This isn't trivial to interpret at first (please read the BioPython manual), but briefly, what is happening is each sequence is being compared _once_ to each other, at each position in the alignment. Changes are counted and then shared evenly across the diagonal; i.e., if there is a difference between a sequence with `A` where another has a `T`, that's `1` change, which would give `0.5` to the `a_t` and `0.5` to the `t_a`. This may seem a little weird, but it is optimised for calculating substitution matrices, which we won't do here but you may need to do. Again, take a look at the BioPython tutorial.

Unsurprisingly, "changes" from `A` to `A`, `C` to `C` and so on, are the most common. This means that the algorithm doesn't even bother, it just counts how many times an `A` "turns" into anything, even if that's another `A`. Weird, but again, this is useful for more advanced calculations that you may need some day.

Of the actual detectable changes, it is also not surprising that the most common changes are between the purines (`A` to `G` or vice versa) or between the pyrimidines (`C` to `T` or vice versa).

---
As I have hinted above, there is a lot more to know about BioPython alignments. May I suggest once more that you take a look at [the BioPython tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html) if you're keen to know more.

In the next lesson, we will use the alignment of mammalian 12S rRNA sequences that we produced here to compute a phylogenetic tree.